In [1]:
>xeus-cling-cpp14

#include <iostream>
#include "demo.cpp"

std::cout << "HELLO C++ 14! " << sum_of_sines(xt::arange<double>(10000.0));

HELLO C++ 14! 1.93951

### Python: pybind11 + xtensor-python

We need to add some wrapper code using pybind11

```cpp
#ifdef XPYTHON
#include "pybind11/pybind11.h"            // Pybind11 import to define Python bindings
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"     // Numpy bindings

PYBIND11_MODULE(xtensor_python_test, m)
{
    xt::import_numpy();
    m.doc() = "Test module for xtensor python bindings";

    m.def("sum_of_sines", sum_of_sines<xt::pyarray<double, xt::layout_type::row_major>>,
                          "Sum the sines of the input values");
}
#endif
```

In [1]:
%%bash
rm -f xpython.cpp

cp demo.cpp xpython.cpp

cat << EOF >> xpython.cpp
#include "pybind11/pybind11.h"            // Pybind11 import to define Python bindings
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"     // Numpy bindings

PYBIND11_MODULE(xpython, m)
{
    xt::import_numpy();
    m.doc() = "Test module for xtensor python bindings";

    m.def("sum_of_sines", sum_of_sines<xt::pyarray<double, xt::layout_type::row_major>>,
                          "Sum the sines of the input values");
}
EOF

In [2]:
>python3
import sys
sys.version

'3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [103]:
%%bash

/usr/bin/g++ -w -shared -fPIC -std=c++14 \
    -lpython3.6m \
    -L/Users/scorlay/miniconda3/python.app/Contents/lib \
    -I/Users/scorlay/miniconda3/include/ \
    -I/Users/scorlay/miniconda3/include/python3.6m \
    -I/Users/scorlay/miniconda3/lib/python3.6/site-packages/numpy/core/include/ \
    xpython.cpp -o xtensor_python_test.so

In [104]:
>python3

import numpy as np
from xtensor_python_test import sum_of_sines

x = np.arange(10000.0)
print("NumPy  : ", np.sum(np.sin(x)))
print("xtensor: ", sum_of_sines(x))

ImportError: dynamic module does not define module export function (PyInit_xtensor_python_test)

### R: Rcpp + xtensor-r

Again, a couple of lines of wrapper code.

```c++
#include <xtensor-r/rarray.hpp>       // R bindings
#include <Rcpp.h>

using namespace Rcpp;

// [[Rcpp::plugins(cpp14)]]
// [[Rcpp::export]]
double sum_of_sines(const xt::rarray<double>& e)
{
    return sum_of_sines_impl(e);
}
#endif
```

In [ ]:
%%bash
rm -f xr.cpp
cp demo.cpp xr.cpp

cat << EOF >> xr.cpp
#include <xtensor-r/rarray.hpp>       // R bindings
#include <Rcpp.h>
using namespace Rcpp;
// [[Rcpp::plugins(cpp14)]]
// [[Rcpp::export]]
double rsum_of_sines(const xt::rarray<double>& e) {
    return sum_of_sines(e);
}
EOF

In [1]:
>ir
Sys.setenv(PKG_CXXFLAGS="-I/usr/local/include/")
Rcpp::sourceCpp("xr.cpp");

arr <- seq(0., 10000.0 - 1.0, 1.0)
rsum_of_sines(arr)

[1] 1.939505

### Julia: CxxWrap + xtensor-julia

Again, a few lines of code.

```c++
#include "jlcxx/jlcxx.hpp"               // libcxxwrap import to define Julia bindings
#include "xtensor-julia/jlarray.hpp"     // Import the jltensor container definition

JULIA_CPP_MODULE_BEGIN(registry)
    jlcxx::Module& mod = registry.create_module("xtjl");
    mod.method("sum_of_sines", sum_of_sines);
JULIA_CPP_MODULE_END
```

In [6]:
%%bash
rm -f xjulia.cpp
cp demo.cpp xjulia.cpp

cat << EOF >> xjulia.cpp
#include "jlcxx/jlcxx.hpp"
#include "xtensor-julia/jlarray.hpp"

JULIA_CPP_MODULE_BEGIN(registry)
    jlcxx::Module& mod = registry.create_module("xjulia");
    mod.method("sum_of_sines", sum_of_sines<xt::jlarray<double>>);
JULIA_CPP_MODULE_END

EOF

In [7]:
%%bash

/usr/bin/g++ -w -shared -fPIC \
    -I/Users/scorlay/.julia/v0.6/CxxWrap/deps/usr/include/ -I/usr/include/julia/ \
    -DJULIA_ENABLE_THREADING -ldl -lpthread \
    -L/Users/scorlay/.julia/v0.6/CxxWrap/deps/usr/ -lcxxwrap_julia \
    xjulia.cpp -o xjulia.so

In [1]:
>julia-0.6

using CxxWrap
wrap_modules(joinpath(".", "xjulia"))

xjulia.sum_of_sines(collect(0.0:1.0:9999))

1.9395054106806988